In [1]:
using LinearAlgebra;
using GLMakie;
using Observables;

In [2]:
const N = 96;
const L = 100;

In [3]:
function get_grid()
	Lend = (N-1) * L / (2 * N)
	li = range(-Lend, Lend; length=N)
    [[(x, y, z)[i] for x in li, y in li, z in li] for i in 1:3]
end
function get_R(grid, ctr)
	function dif_one(agrid, actr)
		di = abs.(agrid .- actr)
		(min.(di, L .- di)).^2
	end
	sqrt.(sum([
		dif_one(agrid, actr) for (agrid, actr) in zip(grid, ctr)
	]))
end

get_R (generic function with 1 method)

In [68]:
function epsilon_for_spin(spin)
	epsilon = [exp(1im * asin(spin)), 0, 1]
	epsilon ./ norm(epsilon)
end

epsilon_for_spin (generic function with 1 method)

In [69]:
function soliton_profile(R, amp, λ, p)
	R² = R.^2;
	amp .* sqrt.(
		(0.998309 ./ (1 .+ 0.0376534 * amp .* R²).^8) .+
		((0.0890172 * amp * λ * p * amp .* R²) ./ (1 .+ 0.0502554 * amp .* R²).^8)
	)
end

soliton_profile (generic function with 1 method)

In [70]:
using ColorSchemes;
using Colors;
function transparent_viridis(base = 0)
	viridis = [cgrad(:viridis)...];
	cgrad([RGBA(c.r, c.g, c.b, clamp(base + i/((1-base) * 256), 0, 1)) for (i, c) in enumerate(viridis)])
end

transparent_viridis (generic function with 2 methods)

In [75]:
function generate(savename, epsilon)
	epsilon = epsilon / norm(epsilon)
	flatten = vec
	grid = get_grid()
	R = get_R(grid, [0, 0, 0])
	prof = soliton_profile(R, 0.04, 0, 0)
	Psi3 = [prof .* ep for ep in epsilon]


	factor = 16
	function downscaled(Arr)
		function downscaled_1D(Arr, dim)
			dimsize = size(Arr, dim)
			outlen = Int(dimsize / factor)
			out = Nothing
			for j in 1:factor
				indexer = [range(;start=1, stop=size(Arr, i), step=1) for i in 1:ndims(Arr)]
				indexer[dim] = range(;start=j, length=outlen, step=factor)
				if out == Nothing
					out = Arr[indexer...]
				else
					out += Arr[indexer...]
				end
			end
			out ./ factor
		end
		nd = ndims(Arr)
		for i in 1:nd
			Arr = downscaled_1D(Arr, i)
		end
		Arr
	end
	
	gridpoints = [Point3f(p) for p in zip([flatten(downscaled(gra)) for gra in grid]...)]

	field_value_vec = Observable(similar(gridpoints, Point3f))
	field_value_dns = Observable(similar(gridpoints, Float32))
	
    # fig = with_theme(theme_black()) do
		fig = Figure(resolution=(800, 800))
		tvcmap = transparent_viridis(0)
		Lend = (N-1) * L / (2 * N)
		axes_limit = (-Lend, Lend)
		axes_limits = (axes_limit, axes_limit, axes_limit)
		field_ax = Axis3(fig[1,1],
			# title=L"\Re[\Psi]",
			limits=axes_limits,
			# xlabel=L"x",
			# ylabel=L"y",
			# zlabel=L"z",
			# elevation=-0.2*π,
			# azimuth=0.25*π,
			# azimuth=0,
			# viewmode=:fit,
		)
		# hidespines!(field_ax)
		hidedecorations!(field_ax)
		arrows!(field_ax, gridpoints, field_value_vec,
			color=field_value_dns,
			arrowcolor=field_value_dns,
			linewidth=1.0, arrowsize=4.0,
			lengthscale=6,
			# align=:center,
			colormap=tvcmap,
			quality=128,
			transparency=true,
			ssao=true,
		)
		fig
	# end
	function scaled_field(p)
		# s = norm(p)
		# l = s * 1E3
		p .* 1E3
	end
	rho = sum(Psi -> abs2.(Psi), Psi3)
	rho_ceiling = log1p(maximum(rho) * 1E6)
	record(fig, savename) do rec_io
		for t in 0:0.1:(2*π)
			fvv = [
				scaled_field(collect(p))
				for p in 
				zip([
					real.(
						flatten(downscaled(Psi)) .* exp(-im * t)
					) for Psi in Psi3
				]...)
			]
			field_value_vec[] = fvv
			rho = downscaled(sum(Psi -> abs2.(Psi), Psi3))
			lpr = log1p.(rho * 1E6) / rho_ceiling
			field_value_dns[] = flatten(lpr)
            recordframe!(rec_io)
		end
	end
end

generate (generic function with 2 methods)

In [76]:
generate("visualization-s=0.0.gif", [0, 0, 1])
generate("visualization-s=1.0.gif", [1, 1im, 0])

"visualization-s=1.0.gif"